In [3]:
from langchain.chains.openai_functions import create_structured_output_runnable
# from langchain_community.chat_models import ChatOllama

# llm = ChatOllama(model='mistral:7b-instruct-v0.2-q6_K', temperature=0.1)

from langchain_experimental.llms.ollama_functions import OllamaFunctions

# ollama_func = OllamaFunctions(llm = llm)

llm = OllamaFunctions(model='mistral:7b-instruct-v0.2-q6_K')

# out = create_structured_output_runnable(ExecuteCode, ollama_func, prompt)

In [4]:
from typing import Optional

from langchain.chains import create_structured_output_runnable
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate


class RecordDog(BaseModel):
    '''Record some identifying information about a dog.'''

    name: str = Field(..., description="The dog's name")
    color: str = Field(..., description="The dog's color")
    fav_food: Optional[str] = Field(None, description="The dog's favorite food")

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an extraction algorithm. Please extract every possible instance"),
        ('human', '{input}')
    ]
)


structured_llm = create_structured_output_runnable(
    RecordDog,
    llm,
    mode="openai-functions",
    enforce_function_usage=True,
    return_single=True
)

print(structured_llm)

structured_llm.invoke({"input": "Harry was a chubby brown beagle who loved chicken"})
# -> RecordDog(name="Harry", color="brown", fav_food="chicken")

first=RunnableBinding(bound=OllamaFunctions(llm=ChatOllama(model='mistral:7b-instruct-v0.2-q6_K', format='json'), tool_system_prompt_template='You have access to the following tools:\n\n{tools}\n\nYou must always select one of the above tools and respond with only a JSON object matching the following schema:\n\n{{\n  "tool": <name of the selected tool>,\n  "tool_input": <parameters for the selected tool, matching the tool\'s JSON schema>\n}}\n'), kwargs={'functions': [{'name': '_OutputFormatter', 'description': 'Output formatter. Should always be used to format your response to the user.', 'parameters': {'type': 'object', 'properties': {'output': {'description': 'Record some identifying information about a dog.', 'type': 'object', 'properties': {'name': {'description': "The dog's name", 'type': 'string'}, 'color': {'description': "The dog's color", 'type': 'string'}, 'fav_food': {'description': "The dog's favorite food", 'type': 'string'}}, 'required': ['name', 'color']}}, 'required': 

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

In [ ]:
first=RunnableBinding(bound=OllamaFunctions(llm=ChatOllama(model='mistral:7b-instruct-v0.2-q6_K', temperature=0.1), tool_system_prompt_template='You have access to the following tools:\n\n{tools}\n\nYou must always select one of the above tools and respond with only a JSON object matching the following schema:\n\n{{\n  "tool": <name of the selected tool>,\n  "tool_input": <parameters for the selected tool, matching the tool\'s JSON schema>\n}}\n'), kwargs={'functions': [{'name': '_OutputFormatter', 'description': 'Output formatter. Should always be used to format your response to the user.', 'parameters': {'type': 'object', 'properties': {'output': {'description': 'Record some identifying information about a dog.', 'type': 'object', 'properties': {'name': {'description': "The dog's name", 'type': 'string'}, 'color': {'description': "The dog's color", 'type': 'string'}, 'fav_food': {'description': "The dog's favorite food", 'type': 'string'}}, 'required': ['name', 'color']}}, 'required': ['output']}}], 'function_call': {'name': '_OutputFormatter'}}) last=PydanticAttrOutputFunctionsParser(pydantic_schema=<class 'langchain.chains.structured_output.base._create_openai_functions_structured_output_runnable.<locals>._OutputFormatter'>, attr_name='output')

In [ ]:
{'functions': [
    {'name': '_OutputFormatter', 
                'description': 'Output formatter. Should always be used to format your response to the user.', 
                'parameters': {'type': 'object', 
                               'properties': {'output': 
                                   {'description': 'Record some identifying information about a dog.', 
                                    'type': 'object', 
                                    'properties': {'name': 
                                        {'description': "The dog's name", 
                                         'type': 'string'}, 
                                        'color': {'description': "The dog's color", 
                                                  'type': 'string'}, 
                                        'fav_food': {'description': "The dog's favorite food", 'type': 'string'}}, 'required': ['name', 'color']}}, 'required': ['output']}}], 'function_call': {'name': '_OutputFormatter'}}) 

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

_SYSTEM_PROMPT = """Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
ExecuteCode({{code: "37593 * 67"}})
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
ExecuteCode({{code: "37593**(1/5)"}})
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718
"""

prompt = ChatPromptTemplate.from_messages(
        [
            ("system", _SYSTEM_PROMPT),
            ("user", "{problem}"),
            MessagesPlaceholder(variable_name="context", optional=True),
        ]
    )

class ExecuteCode(BaseModel):
    """The input to the numexpr.evaluate() function."""

    reasoning: str = Field(
        ...,
        description="The reasoning behind the code expression, including how context is included, if applicable.",
    )

    code: str = Field(
        ...,
        description="The simple code expresssion to execute by numexpr.evaluate().",
    )

In [9]:
extractor = create_structured_output_runnable(ExecuteCode, llm, prompt, 
                                                mode='openai-functions',
                                                enforce_function_usage=True)
from pprint import pprint
pprint(extractor)


ChatPromptTemplate(input_variables=['problem'], input_types={'context': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'context': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\nExecuteCode({{code: "37593 * 67"}})\n...numexpr.evaluate("37593 * 67")...\n```output\n

In [ ]:
ChatPromptTemplate(
    input_variables=['problem'], 
    input_types={'context': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, 
    partial_variables={'context': []}, 
    messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], 
                                                                template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\nExecuteCode({{code: "37593 * 67"}})\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\nExecuteCode({{code: "37593**(1/5)"}})\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n')), 
              HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['problem'], template='{problem}')), 
              MessagesPlaceholder(variable_name='context', optional=True)])
| 
RunnableBinding(bound=ChatOllama(model='mistral:7b-instruct-v0.2-q6_K', temperature=0.1), 
                kwargs={'functions': 
                    [
                        {
                      'name': '_OutputFormatter', 
                      'description': 'Output formatter. Should always be used to format your response to the user.', 
                      'parameters': {
                          'type': 'object', 
                                     'properties': {
                                         'output': {
                                             'description': 'The input to the numexpr.evaluate() function.',
                                             'type': 'object', 
                                             'properties': {
                                                'reasoning': 
                                                     {
                                                         'description': 'The reasoning behind the code expression, including how context is included, if applicable.', 
                                                         'type': 'string'
                                                         },                                          
                                                'code': 
                                                    {
                                                        'description': 'The simple code expresssion to execute by numexpr.evaluate().', 
                                                        'type': 'string'
                                                        }
                                                    },
                                            'required': ['reasoning', 'code']
                                            }
                                         },
                                    'required': ['output']
                                    }
                      }
                     ], 
                    'function_call': {'name': '_OutputFormatter'}})

| PydanticAttrOutputFunctionsParser(pydantic_schema=<class 'langchain.chains.structured_output.base._create_openai_functions_structured_output_runnable.<locals>._OutputFormatter'>, attr_name='output')

In [ ]:
from langchain.chains import create_extraction_chain

Trial

In [15]:
import math
import re
from typing import List, Optional

import numexpr
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_community.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOllama

from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import StructuredTool

_MATH_DESCRIPTION = (
    "math(problem: str, context: Optional[list[str]]) -> float:\n"
    " - Solves the provided math problem.\n"
    ' - `problem` can be either a simple math problem (e.g. "1 + 3") or a word problem (e.g. "how many apples are there if there are 3 apples and 2 apples").\n'
    " - You cannot calculate multiple expressions in one call. For instance, `math('1 + 3, 2 + 4')` does not work. "
    "If you need to calculate multiple expressions, you need to call them separately like `math('1 + 3')` and then `math('2 + 4')`\n"
    " - Minimize the number of `math` actions as much as possible. For instance, instead of calling "
    '2. math("what is the 10% of $1") and then call 3. math("$1 + $2"), '
    'you MUST call 2. math("what is the 110% of $1") instead, which will reduce the number of math actions.\n'
    # Context specific rules below
    " - You can optionally provide a list of strings as `context` to help the agent solve the problem. "
    "If there are multiple contexts you need to answer the question, you can provide them as a list of strings.\n"
    " - `math` action will not see the output of the previous actions unless you provide it as `context`. "
    "You MUST provide the output of the previous actions as `context` if you need to do math on it.\n"
    " - You MUST NEVER provide `search` type action's outputs as a variable in the `problem` argument. "
    "This is because `search` returns a text blob that contains the information about the entity, not a number or value. "
    "Therefore, when you need to provide an output of `search` action, you MUST provide it as a `context` argument to `math` action. "
    'For example, 1. search("Barack Obama") and then 2. math("age of $1") is NEVER allowed. '
    'Use 2. math("age of Barack Obama", context=["$1"]) instead.\n'
    " - When you ask a question about `context`, specify the units. "
    'For instance, "what is xx in height?" or "what is xx in millions?" instead of "what is xx?"\n'
)


_SYSTEM_PROMPT = """Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
ExecuteCode({{code: "37593 * 67"}})
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
ExecuteCode({{code: "37593**(1/5)"}})
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718
"""

_ADDITIONAL_CONTEXT_PROMPT = """The following additional context is provided from other functions.\
    Use it to substitute into any ${{#}} variables or other words in the problem.\
    \n\n${context}\n\nNote that context varibles are not defined in code yet.\
You must extract the relevant numbers and directly put them in code."""


class ExecuteCode(BaseModel):
    """The input to the numexpr.evaluate() function."""

    reasoning: str = Field(
        ...,
        description="The reasoning behind the code expression, including how context is included, if applicable.",
    )

    code: str = Field(
        ...,
        description="The simple code expresssion to execute by numexpr.evaluate().",
    )


def _evaluate_expression(expression: str) -> str:
    try:
        local_dict = {"pi": math.pi, "e": math.e}
        output = str(
            numexpr.evaluate(
                expression.strip(),
                global_dict={},  # restrict access to globals
                local_dict=local_dict,  # add common mathematical functions
            )
        )
    except Exception as e:
        raise ValueError(
            f'Failed to evaluate "{expression}". Raised error: {repr(e)}.'
            " Please try again with a valid numerical expression"
        )

    # Remove any leading and trailing brackets from the output
    return re.sub(r"^\[|\]$", "", output)


def get_math_tool(llm: ChatOllama):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", _SYSTEM_PROMPT),
            ("user", "{problem}"),
            MessagesPlaceholder(variable_name="context", optional=True),
        ]
    )
    extractor = create_structured_output_runnable(ExecuteCode, llm, prompt, mode='openai-functions')

    def calculate_expression(
        problem: str,
        context: Optional[List[str]] = None,
        config: Optional[RunnableConfig] = None,
    ):
        chain_input = {"problem": problem}
        if context:
            context_str = "\n".join(context)
            if context_str.strip():
                context_str = _ADDITIONAL_CONTEXT_PROMPT.format(
                    context=context_str.strip()
                )
                chain_input["context"] = [SystemMessage(content=context_str)]
        code_model = extractor.invoke(chain_input, config)
        try:
            return _evaluate_expression(code_model.code)
        except Exception as e:
            return repr(e)

    return StructuredTool.from_function(
        name="math",
        func=calculate_expression,
        description=_MATH_DESCRIPTION,
    )


from dotenv import load_dotenv
load_dotenv()

from langchain_community.chat_models import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
# from LLMCompiler.math_tools import get_math_tool

from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model='mistral:7b-instruct-v0.2-q6_K', temperature=0.1)

from langchain_experimental.llms.ollama_functions import OllamaFunctions

ollama_func = OllamaFunctions(llm = llm, tool_system_prompt_template='')


search = TavilySearchResults(
    max_results=1,
    description='tavily_search_results_json(query="the search query") - a search engine.',
)



In [16]:
ollama_f = ollama_func.bind(functions = 
                    [
                        {
                      'name': '_OutputFormatter', 
                      'description': 'Output formatter. Should always be used to format your response to the user.', 
                      'parameters': {
                          'type': 'object', 
                                     'properties': {
                                         'output': {
                                             'description': 'The input to the numexpr.evaluate() function.',
                                             'type': 'object', 
                                             'properties': {
                                                'reasoning': 
                                                     {
                                                         'description': 'The reasoning behind the code expression, including how context is included, if applicable.', 
                                                         'type': 'string'
                                                         },                                          
                                                'code': 
                                                    {
                                                        'description': 'The simple code expresssion to execute by numexpr.evaluate().', 
                                                        'type': 'string'
                                                        }
                                                    },
                                            'required': ['reasoning', 'code']
                                            }
                                         },
                                    'required': ['output']
                                    }
                      }
                     ], 
                    function_call= {'name': '_OutputFormatter'})

In [17]:
calculate = get_math_tool(ollama_f)  

tools = [search, calculate]

In [18]:
ollama_f

RunnableBinding(bound=OllamaFunctions(llm=ChatOllama(model='mistral:7b-instruct-v0.2-q6_K', temperature=0.1), tool_system_prompt_template='You have access to the following tools:\n\n{tools}\n\nYou must always select one of the above tools and respond with only a JSON object matching the following schema:\n\n{{\n  "tool": <name of the selected tool>,\n  "tool_input": <parameters for the selected tool, matching the tool\'s JSON schema>\n}}\n'), kwargs={'functions': [{'name': '_OutputFormatter', 'description': 'Output formatter. Should always be used to format your response to the user.', 'parameters': {'type': 'object', 'properties': {'output': {'description': 'The input to the numexpr.evaluate() function.', 'type': 'object', 'properties': {'reasoning': {'description': 'The reasoning behind the code expression, including how context is included, if applicable.', 'type': 'string'}, 'code': {'description': 'The simple code expresssion to execute by numexpr.evaluate().', 'type': 'string'}},

In [19]:
calculate.invoke(
    {
        "problem": "What's the temp of sf + 5?",
        "context": ["Thet empreature of sf is 33 degrees"],
    }
)

ValueError: "mistral:7b-instruct-v0.2-q6_K" did not respond with valid JSON. Please try again.

In [20]:
create_structured_output_runnable.

<function langchain.chains.structured_output.base.create_structured_output_runnable(output_schema: Union[Dict[str, Any], Type[pydantic.main.BaseModel]], llm: langchain_core.runnables.base.Runnable, prompt: Optional[langchain_core.prompts.base.BasePromptTemplate] = None, *, output_parser: Union[langchain_core.output_parsers.base.BaseOutputParser, langchain_core.output_parsers.base.BaseGenerationOutputParser, NoneType] = None, enforce_function_usage: bool = True, return_single: bool = True, mode: Literal['openai-functions', 'openai-tools', 'openai-json'] = 'openai-functions', **kwargs: Any) -> langchain_core.runnables.base.Runnable>

# structured runnable

In [1]:
import json
from typing import Any, Callable, Dict, Literal, Optional, Sequence, Type, Union

from langchain_core.output_parsers import (
    BaseGenerationOutputParser,
    BaseOutputParser,
    JsonOutputParser,
)
from langchain_core.output_parsers.openai_functions import (
    JsonOutputFunctionsParser,
    PydanticAttrOutputFunctionsParser,
    PydanticOutputFunctionsParser,
)
from langchain_core.prompts import BasePromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import Runnable
from langchain_core.utils.function_calling import (
    convert_to_openai_function,
    convert_to_openai_tool,
)

from langchain.output_parsers import (
    JsonOutputKeyToolsParser,
    PydanticOutputParser,
    PydanticToolsParser,
)


In [2]:
def get_openai_output_parser(
    functions: Sequence[Union[Dict[str, Any], Type[BaseModel], Callable]],
) -> Union[BaseOutputParser, BaseGenerationOutputParser]:
    """Get the appropriate function output parser given the user functions.

    Args:
        functions: Sequence where element is a dictionary, a pydantic.BaseModel class,
            or a Python function. If a dictionary is passed in, it is assumed to
            already be a valid OpenAI function.

    Returns:
        A PydanticOutputFunctionsParser if functions are Pydantic classes, otherwise
            a JsonOutputFunctionsParser. If there's only one function and it is
            not a Pydantic class, then the output parser will automatically extract
            only the function arguments and not the function name.
    """
    if isinstance(functions[0], type) and issubclass(functions[0], BaseModel):
        if len(functions) > 1:
            pydantic_schema: Union[Dict, Type[BaseModel]] = {
                convert_to_openai_function(fn)["name"]: fn for fn in functions
            }
        else:
            pydantic_schema = functions[0]
        output_parser: Union[
            BaseOutputParser, BaseGenerationOutputParser
        ] = PydanticOutputFunctionsParser(pydantic_schema=pydantic_schema)
    else:
        output_parser = JsonOutputFunctionsParser(args_only=len(functions) <= 1)
    return output_parser


In [3]:
def create_openai_fn_runnable(
    functions: Sequence[Union[Dict[str, Any], Type[BaseModel], Callable]],
    llm: Runnable,
    prompt: Optional[BasePromptTemplate] = None,
    *,
    enforce_single_function_usage: bool = True,
    output_parser: Optional[Union[BaseOutputParser, BaseGenerationOutputParser]] = None,
    **llm_kwargs: Any,
) -> Runnable:
    
    """Create a runnable sequence that uses OpenAI functions."""

    if not functions:
        raise ValueError("Need to pass in at least one function. Received zero.")
    
    openai_functions = [convert_to_openai_function(f) for f in functions]
    llm_kwargs_: Dict[str, Any] = {"functions": openai_functions, **llm_kwargs}
    
    if len(openai_functions) == 1 and enforce_single_function_usage:
        llm_kwargs_["function_call"] = {"name": openai_functions[0]["name"]}
        
    output_parser = output_parser or get_openai_output_parser(functions)
    
    if prompt:
        return prompt | llm.bind(**llm_kwargs_) | output_parser
    else:
        return llm.bind(**llm_kwargs_) | output_parser


In [4]:
def _create_openai_functions_structured_output_runnable(
    output_schema: Union[Dict[str, Any], Type[BaseModel]],
    llm: Runnable,
    prompt: Optional[BasePromptTemplate] = None,
    *,
    output_parser: Optional[Union[BaseOutputParser, BaseGenerationOutputParser]] = None,
    **llm_kwargs: Any,
) -> Runnable:
    if isinstance(output_schema, dict):
        function: Any = {
            "name": "output_formatter",
            "description": (
                "Output formatter. Should always be used to format your response to the"
                " user."
            ),
            "parameters": output_schema,
        }
    else:

        class _OutputFormatter(BaseModel):
            """Output formatter. Should always be used to format your response to the user."""  # noqa: E501

            output: output_schema  # type: ignore

        function = _OutputFormatter
        output_parser = output_parser or PydanticAttrOutputFunctionsParser(
            pydantic_schema=_OutputFormatter, attr_name="output"
        )
    return create_openai_fn_runnable(
        [function],
        llm,
        prompt=prompt,
        output_parser=output_parser,
        **llm_kwargs,
    )


In [5]:

def create_structured_output_runnable(
    output_schema: Union[Dict[str, Any], Type[BaseModel]],
    llm: Runnable,
    prompt: Optional[BasePromptTemplate] = None,
    *,
    output_parser: Optional[Union[BaseOutputParser, BaseGenerationOutputParser]] = None,
    enforce_function_usage: bool = True,
    return_single: bool = True,
    mode: Literal[
        "openai-functions", "openai-tools", "openai-json"
    ] = "openai-functions",
    **kwargs: Any,
) -> Runnable:
    """Create a runnable for extracting structured outputs."""

    # for backwards compatibility
    force_function_usage = kwargs.get(
        "enforce_single_function_usage", enforce_function_usage
    )

    if mode == "openai-functions":
        return _create_openai_functions_structured_output_runnable(
            output_schema,
            llm,
            prompt=prompt,
            output_parser=output_parser,
            enforce_single_function_usage=force_function_usage,
            **kwargs,  # llm-specific kwargs
        )
    else:
        raise ValueError(
            f"Invalid mode {mode}. Expected one of 'openai-tools', 'openai-functions', "
            f"'openai-json'."
        )


In [9]:
from typing import Optional

from langchain.chains import create_structured_output_runnable
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate


class RecordDog(BaseModel):
    '''Record some identifying information about a dog.'''

    name: str = Field(..., description="The dog's name")
    color: str = Field(..., description="The dog's color")
    fav_food: Optional[str] = Field(None, description="The dog's favorite food")

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an extraction algorithm. Please extract every possible instance"),
        ('human', '{input}')
    ]
)

from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model='mistral:7b-instruct-v0.2-q6_K', temperature=0.1)

from langchain_experimental.llms.ollama_functions import OllamaFunctions

ollama_func = OllamaFunctions(llm = llm, tool_system_prompt_template='')

structured_llm = create_structured_output_runnable(
    RecordDog,
    ollama_func,
    prompt,
    mode="openai-functions",
    enforce_function_usage=True,
    return_single=True
)

print(structured_llm)

structured_llm.invoke({"input": "Harry was a chubby brown beagle who loved chicken"})
# -> RecordDog(name="Harry", color="brown", fav_food="chicken")

first=ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an extraction algorithm. Please extract every possible instance')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]) middle=[RunnableBinding(bound=OllamaFunctions(llm=ChatOllama(model='mistral:7b-instruct-v0.2-q6_K', temperature=0.1), tool_system_prompt_template='You have access to the following tools:\n\n{tools}\n\nYou must always select one of the above tools and respond with only a JSON object matching the following schema:\n\n{{\n  "tool": <name of the selected tool>,\n  "tool_input": <parameters for the selected tool, matching the tool\'s JSON schema>\n}}\n'), kwargs={'functions': [{'name': '_OutputFormatter', 'description': 'Output formatter. Should always be used to format your response to the user.', 'parameters': {'type': 'object', 'properties': {'output': {'description': 'Record 

ValueError: "mistral:7b-instruct-v0.2-q6_K" did not respond with valid JSON. Please try again.

In [12]:
gg =_create_openai_functions_structured_output_runnable(
            RecordDog,
            ollama_func,
            prompt,
            # output_parser=output_parser,
            # enforce_single_function_usage=force_function_usage,
            # **kwargs,  # llm-specific kwargs
        )
# gg.invoke({"input": "Harry was a chubby brown beagle who loved chicken"})
gg
# RecordDog,
#     ollama_func,
#     prompt,
#     mode="openai-functions",
#     enforce_function_usage=True,
#     return_single=True

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an extraction algorithm. Please extract every possible instance')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| RunnableBinding(bound=OllamaFunctions(llm=ChatOllama(model='mistral:7b-instruct-v0.2-q6_K', temperature=0.1), tool_system_prompt_template='You have access to the following tools:\n\n{tools}\n\nYou must always select one of the above tools and respond with only a JSON object matching the following schema:\n\n{{\n  "tool": <name of the selected tool>,\n  "tool_input": <parameters for the selected tool, matching the tool\'s JSON schema>\n}}\n'), kwargs={'functions': [{'name': '_OutputFormatter', 'description': 'Output formatter. Should always be used to format your response to the user.', 'parameters': {'type': 'object', 'properties': {'output': {'description': 'Record some identif

In [16]:
class _OutputFormatter(BaseModel):
    """Output formatter. Should always be used to format your response to the user."""  # noqa: E501

    output: RecordDog  # type: ignore

function = _OutputFormatter
output_parser = PydanticAttrOutputFunctionsParser(
    pydantic_schema=_OutputFormatter, attr_name="output"
)


fn_run = create_openai_fn_runnable(
        [function],
        ollama_func,
        prompt,
        output_parser=output_parser,
        # **llm_kwargs,
)
# fn_run.invoke({"input": "Harry was a chubby brown beagle who loved chicken"})
fn_run
# RecordDog,
#             ollama_func,
#             prompt,

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an extraction algorithm. Please extract every possible instance')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| RunnableBinding(bound=OllamaFunctions(llm=ChatOllama(model='mistral:7b-instruct-v0.2-q6_K', temperature=0.1), tool_system_prompt_template='You have access to the following tools:\n\n{tools}\n\nYou must always select one of the above tools and respond with only a JSON object matching the following schema:\n\n{{\n  "tool": <name of the selected tool>,\n  "tool_input": <parameters for the selected tool, matching the tool\'s JSON schema>\n}}\n'), kwargs={'functions': [{'name': '_OutputFormatter', 'description': 'Output formatter. Should always be used to format your response to the user.', 'parameters': {'type': 'object', 'properties': {'output': {'description': 'Record some identif

In [24]:
# create_openai_fn_runnable
enforce_single_function_usage: bool = True
llm_kwargs: Any

openai_functions = [convert_to_openai_function(f) for f in [function]]
llm_kwargs_: Dict[str, Any] = {"functions": openai_functions}


if len(openai_functions) == 1 and enforce_single_function_usage:
    llm_kwargs_["function_call"] = {"name": openai_functions[0]["name"]}
        
output_parser = output_parser

fn = prompt | llm.bind(**llm_kwargs_) | output_parser

In [32]:
openai_functions = [convert_to_openai_function(f) for f in [function]]

llm_kwargs_["function_call"] = {"name": openai_functions[0]["name"]}

llm_kwargs_

{'functions': [{'name': '_OutputFormatter',
   'description': 'Output formatter. Should always be used to format your response to the user.',
   'parameters': {'type': 'object',
    'properties': {'output': {'description': 'Record some identifying information about a dog.',
      'type': 'object',
      'properties': {'name': {'description': "The dog's name",
        'type': 'string'},
       'color': {'description': "The dog's color", 'type': 'string'},
       'fav_food': {'description': "The dog's favorite food",
        'type': 'string'}},
      'required': ['name', 'color']}},
    'required': ['output']}}],
 'function_call': {'name': '_OutputFormatter'}}

In [25]:
fn

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an extraction algorithm. Please extract every possible instance')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| RunnableBinding(bound=ChatOllama(model='mistral:7b-instruct-v0.2-q6_K', temperature=0.1), kwargs={'functions': [{'name': '_OutputFormatter', 'description': 'Output formatter. Should always be used to format your response to the user.', 'parameters': {'type': 'object', 'properties': {'output': {'description': 'Record some identifying information about a dog.', 'type': 'object', 'properties': {'name': {'description': "The dog's name", 'type': 'string'}, 'color': {'description': "The dog's color", 'type': 'string'}, 'fav_food': {'description': "The dog's favorite food", 'type': 'string'}}, 'required': ['name', 'color']}}, 'required': ['output']}}], 'function_call': {'name': '_Outpu

In [20]:
convert_to_openai_function(function)

{'name': '_OutputFormatter',
 'description': 'Output formatter. Should always be used to format your response to the user.',
 'parameters': {'type': 'object',
  'properties': {'output': {'description': 'Record some identifying information about a dog.',
    'type': 'object',
    'properties': {'name': {'description': "The dog's name", 'type': 'string'},
     'color': {'description': "The dog's color", 'type': 'string'},
     'fav_food': {'description': "The dog's favorite food", 'type': 'string'}},
    'required': ['name', 'color']}},
  'required': ['output']}}

In [26]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

model = OllamaFunctions(model="mistral:7b-instruct-v0.2-q6_K")

model = model.bind(
    functions=[
        {'name': '_OutputFormatter',
 'description': 'Output formatter. Should always be used to format your response to the user.',
 'parameters': {'type': 'object',
  'properties': {'output': {'description': 'Record some identifying information about a dog.',
    'type': 'object',
    'properties': {'name': {'description': "The dog's name", 'type': 'string'},
     'color': {'description': "The dog's color", 'type': 'string'},
     'fav_food': {'description': "The dog's favorite food", 'type': 'string'}},
    'required': ['name', 'color']}},
  'required': ['output']}}
    ],
    function_call={"name": "_OutputFormatter"},
)

In [27]:
fn1 = prompt | model | output_parser

In [28]:
fn

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an extraction algorithm. Please extract every possible instance')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| RunnableBinding(bound=ChatOllama(model='mistral:7b-instruct-v0.2-q6_K', temperature=0.1), kwargs={'functions': [{'name': '_OutputFormatter', 'description': 'Output formatter. Should always be used to format your response to the user.', 'parameters': {'type': 'object', 'properties': {'output': {'description': 'Record some identifying information about a dog.', 'type': 'object', 'properties': {'name': {'description': "The dog's name", 'type': 'string'}, 'color': {'description': "The dog's color", 'type': 'string'}, 'fav_food': {'description': "The dog's favorite food", 'type': 'string'}}, 'required': ['name', 'color']}}, 'required': ['output']}}], 'function_call': {'name': '_Outpu

In [29]:
fn1

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an extraction algorithm. Please extract every possible instance')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| RunnableBinding(bound=OllamaFunctions(llm=ChatOllama(model='mistral:7b-instruct-v0.2-q6_K', format='json'), tool_system_prompt_template='You have access to the following tools:\n\n{tools}\n\nYou must always select one of the above tools and respond with only a JSON object matching the following schema:\n\n{{\n  "tool": <name of the selected tool>,\n  "tool_input": <parameters for the selected tool, matching the tool\'s JSON schema>\n}}\n'), kwargs={'functions': [{'name': '_OutputFormatter', 'description': 'Output formatter. Should always be used to format your response to the user.', 'parameters': {'type': 'object', 'properties': {'output': {'description': 'Record some identifyi

In [30]:
fn1.invoke({"input": "Harry was a chubby brown beagle who loved chicken"})

RecordDog(name='Harry', color='brown', fav_food='chicken')

In [6]:
extractor = create_structured_output_runnable(ExecuteCode, llm, prompt, mode='openai-functions')

NameError: name 'ExecuteCode' is not defined

In [27]:
extractor

ChatPromptTemplate(input_variables=['problem'], input_types={'context': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'context': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\nExecuteCode({{code: "37593 * 67"}})\n...numexpr.evaluate("37593 * 67")...\n```output\n

Trying custom code

In [2]:
from typing import Optional

from langchain.chains import create_structured_output_runnable
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate


class RecordDog(BaseModel):
    '''Record some identifying information about a dog.'''

    name: str = Field(..., description="The dog's name")
    color: str = Field(..., description="The dog's color")
    fav_food: Optional[str] = Field(None, description="The dog's favorite food")

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an extraction algorithm. Please extract every possible instance"),
        ('human', '{input}')
    ]
)

from langchain_experimental.llms.ollama_functions import OllamaFunctions


llm = OllamaFunctions(model='mistral:7b-instruct-v0.2-q6_K')


from helpers.structured_output_runnable_ollama_functions import create_structured_output_runnable_custom


structured_llm = create_structured_output_runnable_custom(
    RecordDog,
    llm,
    prompt,
)

# print(structured_llm)

structured_llm.invoke({"input": "Harry was a chubby brown beagle who loved chicken"})
# -> RecordDog(name="Harry", color="brown", fav_food="chicken")

RecordDog(name='Harry', color='brown', fav_food='chicken')